In [ ]:
%%capture

!git clone https://github.com/hudeven/text
!git clone https://github.com/facebookresearch/pytext


In [ ]:
%%capture

!pip install --upgrade --force-reinstall ./text/stl_text/dataframes/datasets;
!pip install --upgrade --force-reinstall ./text;

!pip install --upgrade --force-reinstall ./pytext;
!pip install --upgrade --force-reinstall tensorboard;

!pip install --upgrade --force-reinstall --pre torch torchtext -f https://download.pytorch.org/whl/nightly/cu101/torch_nightly.html;
!pip install --upgrade --force-reinstall datasets;

!pip install pytorch-lightning

In [1]:
import os

import torch
from pytorch_lightning import Trainer
from stl_text.ops.utils.arrow import convert_csv_to_arrow
from stl_text.datamodule import DocClassificationDataModule
from task import XlmrDocClassificationTask

Install apex from https://github.com/NVIDIA/apex/.


In [8]:
# convert csv to arrow format (only required for the first time)
data_path = "./glue_sst2_tiny"
for split in ("train.tsv", "valid.tsv", "test.tsv"):
    split_path = os.path.join(data_path, split)
    convert_csv_to_arrow(split_path)

# setup datamodule
datamodule = DocClassificationDataModule(data_path=data_path, batch_size=8, drop_last=True)
datamodule.setup("fit")

/Users/stevenliu/opt/anaconda3/envs/fun/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
for i, batch in enumerate(datamodule.train_dataloader()):
    print(f"batch {i}: \n{batch}")

batch 0: 
{'label_id': tensor([1, 1, 0, 0, 1, 1, 0, 0]), 'seq_len': tensor([20, 20, 19, 19, 18, 16, 10, 10]), 'token_ids': tensor([[ 68,  69,  70,  71,  14,  72,  73,  32,  14,  74,  28,  52,   1,  75,
          76,  77,  78,   4,  79,   8],
        [  2, 168,  30, 169, 170, 100, 171,  16, 172, 173,  32,  30, 174,  30,
         175,  30,   4, 176, 177,   8],
        [128,  28,  52, 129, 100,   2, 130,  77,  48, 131, 132, 133,  14, 134,
         135,  28, 136, 137, 128,   0],
        [ 80, 210, 211, 212,  30, 213,   2, 214, 176, 215,  39,  77, 216,  30,
         217, 218,  77, 175,   8,   0],
        [ 28, 178,  35, 179,   4, 153, 180,   2, 181,  47, 182, 183,   1, 184,
         185, 186, 187,   8,   0,   0],
        [ 28, 105, 106,  77,  28, 107, 108,  28,  52, 109,   4, 108,  28, 110,
         111,   8,   0,   0,   0,   0],
        [ 61,  62,  63,  64,   1,  65,  47,  66,  67,   8,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0],
        [128,  28, 127,  18, 206,   2, 207, 

In [ ]:
# construct task
task = XlmrDocClassificationTask(text_transform=datamodule.text_transform, num_class=2,
                                 lr=0.01)

# train model
trainer = Trainer(max_epochs=5, fast_dev_run=True)
trainer.fit(task, datamodule=datamodule)

# export to TorchScript
export_path = "/tmp/doc_task.pt1"
task.to_torchscript("/tmp/doc_task.pt1")
with open(export_path, "rb") as f:
    ts_module = torch.load(f)
    print(ts_module(text_batch=["hello world", "attend is all your need!"]))